In [16]:
import pandas as pd
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import os.path
import pickle

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
def get_google_sheets_service():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '/Users/sangyuxu/Library/Mobile Documents/com~apple~CloudDocs/EspressoManu/Gdocs API/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
            
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('sheets', 'v4', credentials=creds)


In [19]:
def dataframe_to_sheets(df, spreadsheet_id, sheet_name, start_cell='A1', clear_range=False):
    """
    Write a Pandas DataFrame to a specific location in a Google Sheet.
    
    Parameters:
    - df: pandas DataFrame to write
    - spreadsheet_id: ID of the Google Sheet (from the URL)
    - sheet_name: Name of the sheet to write to
    - start_cell: Cell where to start writing (e.g., 'A1', 'B5')
    - clear_range: Whether to clear the range before writing
    """
    service = get_google_sheets_service()
    
    # Convert DataFrame to values list
    values = [df.columns.tolist()] + df.values.tolist()
    
    # Calculate the range
    col = ''.join(filter(str.isalpha, start_cell))
    row = int(''.join(filter(str.isdigit, start_cell)))
    end_col = chr(ord(col) + len(df.columns) - 1)
    end_row = row + len(values) - 1
    range_name = f'{sheet_name}!{start_cell}:{end_col}{end_row}'
    
    # Clear the range if requested
    if clear_range:
        service.spreadsheets().values().clear(
            spreadsheetId=spreadsheet_id,
            range=range_name
        ).execute()
    
    # Write the values
    body = {
        'values': values
    }
    
    result = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=range_name,
        valueInputOption='RAW',
        body=body
    ).execute()
    
    return result


def update_cell(spreadsheet_id, cell_range, value):
    service = get_google_sheets_service()

    body = {
        'values': [[value]]  # Double brackets because we're sending a 2D array
    }
    
    result = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=cell_range,
        valueInputOption='RAW',
        body=body
    ).execute()
    
    return result
def find_last_row_in_column(spreadsheet_id, sheet_name, column='A'):
    service = get_google_sheets_service()
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id,
        range=f'{sheet_name}!{column}:{column}'  # Just look at one column
    ).execute()
    
    values = result.get('values', [])
    
    # Find last non-empty cell
    last_row = 0
    for i, row in enumerate(values):
        if row and row[0].strip():  # Check if cell has non-empty value
            last_row = i + 1
    
    return last_row if last_row > 0 else 1

# Example usage:


NameError: name 'service' is not defined

In [22]:
def write_results_table(SPREADSHEET_ID, title, table, sheet, starting_row = None):
    if starting_row == None:
        last_row = find_last_row_in_column(SPREADSHEET_ID, sheet, 'A')
        print(last_row)
    else:
        last_row = starting_row
    update_cell(SPREADSHEET_ID, sheet+'!A'+str(last_row+2), title )
    dataframe_to_sheets(table.round(2).astype(str), SPREADSHEET_ID, sheet, 'A'+str(last_row+3), clear_range=True)

In [ ]:

def make_delta_results(c, ctrlGeno, testGeno):
    results = c.mean_diff.results[['control','test','control_N','test_N','effect_size','difference','bca_low','bca_high', 'pvalue_permutation']]
    results = results.rename(columns={'control': 'control condition', 'test': 'test condition'})
    results.insert(0, 'test genotype', [testGeno])
    results.insert(0, 'control genotype', [ctrlGeno])
    return results


def make_delta2_results(c, ctrlGeno, testGeno):
    results = c.mean_diff.results[['control','test','control_N','test_N','effect_size','difference','bca_low','bca_high', 'pvalue_permutation']]
    results = results.rename(columns={'control': 'control condition', 'test': 'test condition'})
    # print(results)
    new_row = pd.DataFrame({'control condition': results.loc[0, 'test condition'] + ' - ' + results.loc[0, 'control condition'],
                            'test condition': results.loc[1, 'test condition'] + ' - ' +  results.loc[1, 'control condition'],
                            'effect_size': ['Delta Delta'],
                            'control_N': [5000],
                            'test_N': [5000],
                            'difference': c.mean_diff.delta_delta.difference,
                            'bca_low': c.mean_diff.delta_delta.bca_low,
                            'bca_high': c.mean_diff.delta_delta.bca_high,
                            'pvalue_permutation': c.mean_diff.delta_delta.pvalue_permutation
                           })
    results = pd.concat([results, new_row], ignore_index=True)
    results.insert(0, 'test genotype', [testGeno, testGeno, ''])
    results.insert(0, 'control genotype', [ctrlGeno, ctrlGeno, ''])
    return results

In [18]:
get_google_sheets_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=353793915331-eqquk98jeo8d5s9hi7dn9p934k92m04t.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61748%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=VRL1DkrrYh4Ac7aVbWcpSLYlLEZIFe&access_type=offline
